<h1 align = "center">Επεξεργασία Φωνής και Φυσικής Γλώσσας</h1> 
<h2 align = "center">1η Προπαρασκευαστική Εργασία</h2> 
<h3 align = "center"> Θεoδωρόπουλος Νικήτας -03115185</h3>
<h3 align = "center"> Καλλιώρα Δωροθέα - 03115176</h3>




<h3>Σκοπός:</h3>
Σκοπός της εργαστηριακής άσκησης είναι η δημιουργία ενός απλού ορθογράφου με χρήση μηχανών πεπερασμένων καταστάσεων (fst) με τη βοήθεια της βιβλιοθήκης openfst (v1.6.1). Θα χρησιμοποιήσουμε corpus απο δημόσια διαθέσιμα βιβλία για να δημιουργήσουμε ενα γλωσσικό μοντέλο σε μορφή λεξικού. Τέλος θα γίνει εισαγωγή σε αναπαραστάσεις word2vec. 

***Βήμα 1***

(α)
Κατασκευάζουμε το corpus με σύμπτηξη plain text βιβλίων, δημόσια διαθέσημα στο Project Gutenberg. Θα χρησιμοποιήσουμε corpus των δυο παρακάτω βιβλίων:  
<br>
<div class="image123">
    <div class="imgContainer"  Style = "float:left">
        <p>The War of the Worlds by H. G. Wells</p>
        <img src="./img/book36.jpg" height="50" width="150"/>
    </div>
    <div class="imgContainer" Style = "float:right">
        <p>Grimms' Fairy Tales by Jacob Grimm and Wilhelm Grimm</p>
        <img class="middle-img" src="./img/book2591.jpg"/ height="10" width="150"/>
    </div>
</div>


In [3]:
#!wget -P ./data/ http://www.gutenberg.org/files/36/36-0.txt
#!wget -P ./data/ http://www.gutenberg.org/files/2591/2591-0.txt

***Βήμα 2***  Niki
ΔΙαβαζουμε αρχειο κτλπ...


In [4]:
# Step 2

def identity_preprocess(str):
  return str

def readfile(path,preprocess=identity_preprocess):
  processed_txt=[]
  f = open(path, "r")
  for line in f:
    processed_txt = processed_txt + preprocess(line)
  return processed_txt 

def tokenize(s):
    s = s.strip().lower()
    s = ''.join(c if c.isalpha() else ' ' for c in s)
    # We replace all non-alpha characters with ' '
    s = s.replace('\n',' ')
    s = s.split()
    return s



In [5]:
#Step 2 (d)

#!pip install --user -U nltk
import nltk
nltk.download('punkt')

sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
sentence = """At eight o'clock on Thursday morning....Arthur didn't feel very good.
              He quickly rushed to the Doctor!"""
tokenizers = [tokenize,nltk.word_tokenize,sent_detector.tokenize]
names = ["custom","nltk punkt","nltk sentece detector"]
for tokenizer,name in zip(tokenizers,names):
    print(name,":\n",tokenizer(sentence),'\n')


custom :
 ['at', 'eight', 'o', 'clock', 'on', 'thursday', 'morning', 'arthur', 'didn', 't', 'feel', 'very', 'good', 'he', 'quickly', 'rushed', 'to', 'the', 'doctor'] 

nltk punkt :
 ['At', 'eight', "o'clock", 'on', 'Thursday', 'morning', '...', '.Arthur', 'did', "n't", 'feel', 'very', 'good', '.', 'He', 'quickly', 'rushed', 'to', 'the', 'Doctor', '!'] 

nltk sentece detector :
 ["At eight o'clock on Thursday morning....Arthur didn't feel very good.", 'He quickly rushed to the Doctor!'] 



[nltk_data] Downloading package punkt to
[nltk_data]     /home/dorotheakal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


***Βήμα 2*** 

(δ) 
Για αυτό το ερώτημα επιλέξαμε να χρησιμοποιήσουμε τους παρακάτω δυο tokenizers απο την βιβλιοθήκη nltk καθώς και την δική μας υλοποίηση:

<b>Sentence Tokenizer</b>: Αυτός ο tokenizer χωρίζει το κείμενο σε μία λίστα από προτάσεις χρησιμοποιώντας έναν unsupervised αλγόριθμο για να δημιουργήσει ένα μοντέλο που αναγμωρίζει τις λέξεις με απόστροφο, τον συνδυασμό λέξεων και τις λέξεις που ξεκινούν τις προτάσεις. Ο αλγόριθμος αυτός πρέπει να εκπαιδευτεί σε ένα μεγάλο σύνολο δεδομένων πριν μπορέσει να χρησιμοποιηθεί αποτελεσματικά. Το NLTK data package περιλαμβάνει ένα προ-εκπαιδευμένο Punkt tokenizer για την αγγλική γλώσσα.

<b>Word Τokenizer</b>: Αυτός ο tokenizer εντοπίζει τις λέξεις του κειμένου (ενός string) και τις αποθηκεύει σε μία λίστα. Από τα σημεία στίξης κρατάει μόνο εκείνα που διαχωρίζουν προτάσεις. Ακόμα, όπως και ο Sentence Tokenizer χρησιμοποιεί ένα προ-εκπαιδευμένο Punkt tokenizer για την αγγλική γλώσσα το οποίο δεν χωρίζει τις λέξεις με απόστροφο όπως το didn't. 

<b>Tokenize()</b>: Αυτός είναι ο tokenizer που υλοποιήθηκε στα πλαίσια της άσκησης. Παίρνει σαν είσοδο ένα sting και επιστρέφει μόνο γραμματικούς χαρακτήρες (λέξεις) σε μία λίστα. Αυτό υλοποιέιται αφαιρώντας όλα τα σημεία στίξης, τα κενά, τις αλλαγές γραμμής, τους αριθμούς, και κάνοντας όλα τα γράμματα των λέξεων πεζά. 

Η κύρια διαφορά της tokenize() με τους παραπάνω tokenizers της βιβλιοθήκης nltk είναι ότι η tokenize() αφαιρώντας όλα τα σημεία στίξης, αφαιρεί και τις αποστρόφους από τις λέξεις. Αυτό έχει ως αποτέλεσμα η λέξη "o'clock" να μην εμφανίζεται ως έχει στην τελική λίστα με τις λέξεις του κειμένου, αλλά να διασπάται στις λέξεις "o" και "clock". Όμως, η λέξη "didn" που προκύπτει δεν αποτελεί μία σωστή αγγλική λέξη και άρα δεν θα έπρεπε να υπάρχει στο λεξικό που φτιάχνουμε.   

Εμείς στα πλαίσια της άσκησης λοιπόν, βλέπουμε ότι έχουμε υλοποιήσει μια πολύ απλή έκδοση της tokenize. Ακόμα, συγκρίνοντας την υλοποιήση μας με τις υλοποιήσεις των δύο παραπάνω tokenizers συμπεραίνουμε, ότι ενώ θα μπορούσαμε να διορθώσουμε τον διαχωρισμό των λέξεων με απόστροφο, δεν θα μπορούσαμε εύκολα να εντοπίζουμε τα σημεία στίξης που διαχωρίζουν τις προτάσεις, αφού η Word Tokenizer χρησιμοποιεί έναν προ-εκπαιδευμένο Punkt tokenizer για αυτό το σκοπό.


In [6]:
# Step 3
import random
random.seed()

text_1 = readfile('./data/36-0.txt',tokenize)
text_2 = readfile('./data/2591-0.txt',tokenize)
text = text_1 + text_2

print(len(text),"words overall")
word_corpus = list(set(text))
print(len(word_corpus),"unique words in corpus")

# Convert list of words to list of chars, get unique chars with set
alphabet = list(set([c for word in word_corpus for c in word]))
print(len(alphabet),"symbols in alphabet:")
print(alphabet)



169392 words overall
9288 unique words in corpus
28 symbols in alphabet:
['b', 'm', 'p', 'q', 'o', 'x', 's', 'n', 'l', 'z', 'k', 'w', 'g', 'æ', 'e', 'd', 'a', 'f', 'c', 't', 'i', 'ç', 'h', 'y', 'r', 'u', 'j', 'v']


Με βάση το αλφάβητο που προέκυψε απο το word corpus δημιουργούμε το αρχείο συμβόλων εισόδου και εξόδου για τα fst.

In [7]:
# Step 4

def create_syms(alphabet):
    f = open("./chars.syms","w+")
    f.write(f'<epsilon>     0\n')  #for <epsilon> index 0
    for i in range(len(alphabet)):
        f.write(f'{alphabet[i]}     {i+50}\n')  #for other characters index i+50
        
create_syms(alphabet)
    

***Βήμα 5*** niki

(α)Ως μετρική για την απόσταση 2 λέξεων θα χρησιμοποιήσουμε την απόσταση Levenshtein (ή edit distance). Η απόσταση μπορέι να υπολογιστεί αναδρομικά με χρήση dynamic programming (DP). Ανάμεσα σε δύο λέξεις μπορούν να γίνουν τρείς τύποι μετατροπών: εισαγωγή ενός χαρακτήρα ($ \epsilon \rightarrow a$), μετατροπή ενός χαρακτήρα ($ a \rightarrow b$) και διαγραφή ενός χαρακτήρα ($a \rightarrow \epsilon$). Θεωρούμε αρχικά ίδιο κόστος για όλες τις μετατροπές.

Δημιουργούμε κατάλληλο fst με μία κατάσταση, αντιστοιχίζοντας κάθε χαρακτήρα σε κάθε άλλον και στο $\epsilon$, και το $\epsilon$ σε κάθε χαρακτήρα με βάρος 1. Αντιστοιχίζουμε ακόμα τον χαρακτήρα στον εαυτό του με βάρος 0. Αν πάρουμε το shortest path τότε η έξοδος θα είναι λέξη εισόδου, αφού το ελάχιστο βάρος προκύπτει αν δεν γίνει καμία μετατροπή. 

(β)Στην υλοποίηση που έγινε για το ερώτημα 5 έχουμε υποθέσει ότι όλα τα edits έχουν ίσο βάρος. Αυτή ειναι μια αφελής υπόθεση αφού κάθε είδος λάθους (παράλλειψη, επανάληψη ή λάθος τοποθέτηση γράμματος)  έχει την ίδια πιθανότητα εμφάνισης. Ιδανικά θα θέλαμε να γνωρίζουμε την κατανομή του λάθους, την οποία μπορούμε να εκτιμήσουμε απο ένα σύνολο δεδομένων. Για ενα ζεύγος (σωστή λέξη, λανθασμένη λέξη) μπορούμε να υπολογίσουμε τη διόρθωση που απαιτείται και η πιθανότητα θα προκύψει $ P[ a\rightarrow b] = \frac{|error = b \rightarrow a|}{|errors|}, \forall a,b \in \{A+\epsilon\} $, όπου $A$ το αλφάβητο. Αυτές εινα οι a priori πιθανότητες λαθών με βάση το training data. Αυτές μπορούν να χρησιμοποιηθούν για να υπολιστούν τα βάρη στο μοντέλο αναγνώρισης μας με fst, ώστε να είναι αντιστρόφως ανάλογα της πιθανότητας. Μια καλή συνάρτηση αντιστοίχησης που διατηρεί τις αναλογίες είναι η $\frac{1}{x}$.



In [8]:
#Step 5

# arc format: src dest ilabel olabel [weight]
# final state format: state [weight]
# lines may occur in any order except initial state must be first line
# unspecified weights default to 0.0 (for the library-default Weight type) 

f = open('lev.fst',"w+")

def format_arc(src,dst,src_sym,dst_sym,w,f,acceptor=False):
    if not acceptor:
        f.write("{} {} {} {} {} \n".format(src,dst,src_sym,dst_sym,w))
    else:
        f.write("{} {} {} \n".format(src,dst,src_sym))
letters =  alphabet

for i in range(0, len(letters)):
    format_arc(src=0, dst=0, src_sym="<epsilon>", dst_sym=letters[i], w=1,f=f)
    format_arc(src=0, dst=0, src_sym=letters[i], dst_sym=letters[i], w=0,f=f)
    format_arc(src=0, dst=0, src_sym=letters[i], dst_sym="<epsilon>", w=1,f=f)
    for j in range(0, len(letters)):
        if(j!=i):
            format_arc(src=0, dst=0, src_sym=letters[i], dst_sym=letters[j], w=1,f=f)   
f.write('0\n')
f.close()
    

In [9]:
!fstcompile --isymbols=chars.syms --osymbols=chars.syms  lev.fst lev.bin.fst

***Βήμα 6***

Ένας <b>αποδοχέας(acceptor)</b> είναι ένα fst όπου κάθε μετάβαση έχει ενα label (και προεραιτικά βάρος). Στην βιβλιοθήκη openfst1-6-1 μπορέι να θεωρηθεί ως μετατροπέας(transducer) με ίδιο input και output label.

Σε αυτό το ερώτημα κατασκευάζουμε εναν αποδοχέα που αποδέχεται κάθε λέξη του λεξικού. Ο αποδοχέας έχει μία κοινή αρχική κατάσταση για κάθε λέξη και δεν έχουμε βάρος στις μεταβάσεις (w = 0).

In [10]:
#Step 6

def create_acceptor(words,file):
    f = open(file,"w+")
    s = 0
    final_states = []
    for word in words:
        format_arc(0,s+1,word[0],word[0],0,f,acceptor=True)
        s += 1
        for letter in word[1:]:
            format_arc(s,s+1,letter, letter, 0,f,acceptor=True)
            s += 1
        final_states.append(s)

    for state in final_states:
        f.write(f'{state}\n')
    f.close()    
create_acceptor(word_corpus,"acceptor.fst")

In [11]:
!fstcompile --isymbols=chars.syms --osymbols=chars.syms --acceptor acceptor.fst acceptor.bin.fst
#!fstdraw --isymbols=chars.syms --osymbols=chars.syms -portrait rosebud.bin1.fst | dot -Tjpg >acceptor.jpg

#TODO: Τι κάνουν οι απο κατω συναρτήσεις απο την βιβλιοθήκη..... niki

In [12]:
#Step 6b

!fstrmepsilon acceptor.bin.fst acceptor.bin.fst
!fstdeterminize acceptor.bin.fst acceptor.bin.fst
!fstminimize acceptor.bin.fst acceptor.bin.fst


In [13]:
#Step 7
#mini.fst - lev.fst

!fstarcsort --sort_type=olabel lev.bin.fst lev.bin.fst
!fstarcsort --sort_type=ilabel acceptor.bin.fst acceptor.bin.fst

In [14]:
#Step 7(a)

!fstcompose  lev.bin.fst acceptor.bin.fst spell_checker.bin.fst

In [15]:
#Step 7b

create_acceptor(['cit'],"in.fst")

!fstcompile  --isymbols=chars.syms --osymbols=chars.syms  --acceptor in.fst in.bin.fst
!fstarcsort --sort_type=ilabel spell_checker.bin.fst spell_checker.bin.fst 
!fstarcsort --sort_type=olabel in.bin.fst in.bin.fst 
print("Μια πιθανή πρόβλεψη του min edit spell checker για τη λέξη cit είναι η: ")
!fstcompose in.bin.fst spell_checker.bin.fst |fstshortestpath --nshortest=1 \
| fstrmepsilon |  fsttopsort |fstprint -isymbols=chars.syms  -osymbols=chars.syms\
| cut -f4 | grep -v "<epsilon>" |head -n -1 | tr -d '\n'

Μια πιθανή πρόβλεψη του min edit spell checker για τη λέξη cit είναι η: 
fit

In [16]:
!fstcompose in.bin.fst spell_checker.bin.fst |fstshortestpath --nshortest=1   \
| fstrmepsilon |  fsttopsort |fstprint -isymbols=chars.syms  -osymbols=chars.syms

0	1	c	f	1
1	2	i	i
2	3	t	t
3


In [17]:
#!wget https://raw.githubusercontent.com/georgepar/python-lab/master/spell_checker_test_set -P ./data/

In [18]:
# Step 8:

file = open('./data/spell_checker_test_set',"r")
y_test = []
X_test = []
for line in file:
    [y,X] = line.split(':')
    X = X.split()
    for word in X:
        X_test.append(word)
        y_test.append(y)

print(y_test[0:4])
print(X_test[0:4])
print(len(y_test))
print(len(X_test))
idxs = random.sample(range(0, len(y_test)), 20)


['contented', 'contented', 'contented', 'contented']
['contenpted', 'contende', 'contended', 'contentid']
270
270


In [19]:
def predict(Y,X):
    correct_pred=0
    for y,x in zip(Y,X):
        create_acceptor([x],"input.fst")

        !fstcompile  --isymbols=chars.syms --osymbols=chars.syms  --acceptor input.fst input.bin.fst
        !fstarcsort --sort_type=ilabel spell_checker.bin.fst spell_checker.bin.fst 
        !fstarcsort --sort_type=olabel input.bin.fst input.bin.fst 
        prediction = !fstcompose input.bin.fst spell_checker.bin.fst |fstshortestpath --nshortest=1 \
        | fstrmepsilon |  fsttopsort |fstprint -isymbols=chars.syms  -osymbols=chars.syms\
        | cut -f4 | grep -v "<epsilon>" |head -n -1 | tr -d '\n' 
        print("Wrong = ",x,"-- Correct = ",y,"-- Prediction = ",prediction[0])
        if y == prediction[0]:
            correct_pred+=1
    print(f"Accuracy:{correct_pred/len(Y)}%")


In [20]:
X_rand = [X_test[i] for i in idxs]
Y_rand = [y_test[i] for i in idxs]
predict(Y_rand,X_rand)

Wrong =  specal -- Correct =  special -- Prediction =  special
Wrong =  oppossite -- Correct =  opposite -- Prediction =  opposite
Wrong =  compair -- Correct =  compare -- Prediction =  complain
Wrong =  stanerdizing -- Correct =  standardizing -- Prediction =  standing
Wrong =  contende -- Correct =  contented -- Prediction =  content
Wrong =  curtions -- Correct =  curtains -- Prediction =  curious
Wrong =  scarsely -- Correct =  scarcely -- Prediction =  scarcely
Wrong =  relly -- Correct =  really -- Prediction =  belly
Wrong =  reafreshment -- Correct =  refreshment -- Prediction =  refreshed
Wrong =  pertend -- Correct =  pretend -- Prediction =  extend
Wrong =  possable -- Correct =  possible -- Prediction =  possible
Wrong =  seperate -- Correct =  separate -- Prediction =  separate
Wrong =  refreshmant -- Correct =  refreshment -- Prediction =  refresh
Wrong =  desicate -- Correct =  desiccate -- Prediction =  delicate
Wrong =  parrallel -- Correct =  parallel -- Prediction =

***Βήμα 8***

Σε αυτό το βήμα χρησιμοποιήσαμε το μετατροπέα που υλοποιήσαμε στο βήμα 7 για να διορθώσουμε κάποιες από τις λέξεις του test set. Παραπάνω φαίνεται το αποτέλεσμα για είκοσι τυχαίες λέξεις του test set.

Αρχικά, βλέπουμε ότι το αν θα κάνει τη σωστή διόρθωση ο min edit distance spell checker εξαρτάται και από το αν η σωστή λέξη στην οποία θέλουμε να πάμε υπάρχει στο λεξικό. Αν η λέξη υπάρχει, η πιθανότητα να γίνει η σωστή διόρθωση αυξάνεται. 
Ακόμα, από τα παραπάνω αποτελέσματα παρατηρούμε ότι όταν έχουμε ισοβαρείς μεταβάσεις το αποτέλεσμα δεν είναι καθόλου ικανοποιητικό. Αυτό συμβαίνει γιατί όλα τα edits έχουν ίσο κόστος και άρα η επιλογή της διόρθωσης γίνεται τυχαία. 
Τέλος, παρατηρήσαμε ότι επειδή μπορεί να υπάρχουν πολλές λέξεις με το ίδιο edit distance, η επιλογή της λέξης που τυπώνεται γίνεται πάλι τυχαία. Σε αυτό ωφείλονται πάλι οι ισοβαρείς μεταβάσεις που ορίσαμε στον μετατροπέα του βήματος 7.

In [21]:
#Step 9(a)

def tokenized_list(path,preprocess=identity_preprocess):
  list_of_sentences=[]
  f = open(path, "r")
  for line in f:
    l = preprocess(line)
    if l:
        list_of_sentences.append(l)
  return list_of_sentences


list1 = tokenized_list('./data/36-0.txt',tokenize)
list2 = tokenized_list('./data/2591-0.txt',tokenize)
final_list = list1 + list2 

In [22]:
!pip install -U gensim
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

Requirement already up-to-date: gensim in /home/dorotheakal/anaconda3/lib/python3.7/site-packages (3.8.1)


In [29]:
#Step 9(b,c)

# Initialize word2vec. Context is taken as the 2 previous and 2 next words
model = Word2Vec(final_list, window=5, size=100, workers=4)
model.train(final_list, total_examples=len(final_list), epochs=1000)
# get ordered vocabulary list
voc = model.wv.index2word
# get vector size
dim = model.vector_size
#pick 10 random words from the dictionary
idxs = random.sample(range(0, len(voc)), 10)
rand_words = [voc[i] for i in idxs]
for word in rand_words:
    print(f'Most similar words to "{word}":')
    for word,sim in model.wv.most_similar(word):
        print(f'     "{word}" -- sim: {sim}')


Most similar words to "bell":
     "planted" -- sim: 0.3844985067844391
     "dawned" -- sim: 0.3270135521888733
     "lions" -- sim: 0.321536123752594
     "party" -- sim: 0.3153380751609802
     "waiting" -- sim: 0.3137977421283722
     "drop" -- sim: 0.2890227437019348
     "ulla" -- sim: 0.2797062397003174
     "mr" -- sim: 0.2777710556983948
     "printed" -- sim: 0.27099916338920593
     "sunset" -- sim: 0.26970311999320984
Most similar words to "silver":
     "your" -- sim: 0.33258122205734253
     "cask" -- sim: 0.3314518928527832
     "sense" -- sim: 0.3314470648765564
     "jewels" -- sim: 0.319366991519928
     "formed" -- sim: 0.31553730368614197
     "mighty" -- sim: 0.31182727217674255
     "spun" -- sim: 0.3105581998825073
     "treasure" -- sim: 0.30659621953964233
     "knowledge" -- sim: 0.3021213114261627
     "cake" -- sim: 0.2975206971168518
Most similar words to "window":
     "house" -- sim: 0.4465496838092804
     "room" -- sim: 0.40661484003067017
     "door" -

***Βήμα 9***

(γ)Τα αποτελέσματα του ερωτήματος 9(γ) δεν είναι τόσο ποιοτικά όσο θα περιμέναμε αφού πολλές από τις λέξεις δεν φαίνεται να έχουν πολύ μεγάλο similarity.

Για να προσπαθήσουμε να βελτιώσουμε τα αποτελέσματα:

(i)αυξήσουμε αρχικά το μέγεθος του παραθύρου context κρατώντας τον αριθμό εποχών σταθερό,

(ii)αυξήσουμε τον αριθμό των εποχών κρατώντας τον αριθμό του παραθύρου σταθερό,

(iii)αυξήσουμε και τις δύο τιμές.

Παρακάτω παραθέτουμε τα αποτελέσματα του similarity 20 τυχαίων λέξεων για κάθε μία από τις τρεις περιπτώσεις καθώς και ένα σχολιασμό των αποτελεσμάτων.


In [30]:
#Step 9(c (i))

def similarity(w,s,e):
    # Initialize word2vec. Context is taken as the 2 previous and 2 next words
    model = Word2Vec(final_list, window=w, size=s, workers=4)
    model.train(final_list, total_examples=len(final_list), epochs=e)
    # get ordered vocabulary list
    voc = model.wv.index2word
    # get vector size
    dim = model.vector_size
    #pick 10 random words from the dictionary
    rand_words = [voc[i] for i in idxs]
    for word in rand_words:
        print(f'Most similar words to "{word}":')
        for word,sim in model.wv.most_similar(word):
            print(f'     "{word}" -- sim: {sim}')
            

In [33]:
similarity(10,100,2000)

Most similar words to "bell":
     "hunting" -- sim: 0.32449445128440857
     "grow" -- sim: 0.30320775508880615
     "boiling" -- sim: 0.2744957506656647
     "curdken" -- sim: 0.2740527093410492
     "john" -- sim: 0.2679397761821747
     "chain" -- sim: 0.26712465286254883
     "f" -- sim: 0.2648935914039612
     "party" -- sim: 0.26468372344970703
     "cast" -- sim: 0.2637123167514801
     "cake" -- sim: 0.26236748695373535
Most similar words to "silver":
     "torn" -- sim: 0.3431171178817749
     "miserable" -- sim: 0.3217596709728241
     "gifts" -- sim: 0.32025212049484253
     "train" -- sim: 0.31301024556159973
     "dancing" -- sim: 0.30746889114379883
     "race" -- sim: 0.3057774305343628
     "easy" -- sim: 0.2964330315589905
     "tore" -- sim: 0.29458776116371155
     "rampion" -- sim: 0.29034423828125
     "exactly" -- sim: 0.2902930974960327
Most similar words to "window":
     "fight" -- sim: 0.345350444316864
     "house" -- sim: 0.3443455696105957
     "wall" -- s

In [34]:
similarity(15,100,2000)

Most similar words to "bell":
     "hunting" -- sim: 0.3429935574531555
     "less" -- sim: 0.3279402256011963
     "disclaimer" -- sim: 0.32247552275657654
     "dawned" -- sim: 0.30565589666366577
     "cast" -- sim: 0.2974476218223572
     "mill" -- sim: 0.29500746726989746
     "tan" -- sim: 0.2875020503997803
     "this" -- sim: 0.2817056179046631
     "drank" -- sim: 0.2732568085193634
     "gently" -- sim: 0.2725380063056946
Most similar words to "silver":
     "miserable" -- sim: 0.35244911909103394
     "tore" -- sim: 0.31512022018432617
     "steal" -- sim: 0.3009749948978424
     "lamp" -- sim: 0.29763495922088623
     "train" -- sim: 0.2955177128314972
     "easy" -- sim: 0.2939110994338989
     "full" -- sim: 0.2925071716308594
     "courage" -- sim: 0.2877197861671448
     "liability" -- sim: 0.28388547897338867
     "blackness" -- sim: 0.28297221660614014
Most similar words to "window":
     "western" -- sim: 0.3311654329299927
     "gate" -- sim: 0.32796308398246765
   

In [31]:
#Step 9(c (ii))

similarity(5,100,2000)

Most similar words to "bell":
     "planted" -- sim: 0.3346071243286133
     "waiting" -- sim: 0.29823920130729675
     "party" -- sim: 0.29563140869140625
     "drop" -- sim: 0.2951899766921997
     "support" -- sim: 0.29239869117736816
     "curdken" -- sim: 0.28694701194763184
     "everyone" -- sim: 0.28688186407089233
     "breakfast" -- sim: 0.2795769274234772
     "steak" -- sim: 0.2774592339992523
     "dawned" -- sim: 0.27737024426460266
Most similar words to "silver":
     "smart" -- sim: 0.32704150676727295
     "sense" -- sim: 0.3109772205352783
     "treasure" -- sim: 0.30615323781967163
     "your" -- sim: 0.30246713757514954
     "steak" -- sim: 0.29568785429000854
     "buttons" -- sim: 0.2930767238140106
     "locks" -- sim: 0.28750860691070557
     "yard" -- sim: 0.2864888310432434
     "incredible" -- sim: 0.2850775718688965
     "torn" -- sim: 0.28429627418518066
Most similar words to "window":
     "house" -- sim: 0.4080429971218109
     "gates" -- sim: 0.366605579

In [32]:
similarity(5,100,3000)

Most similar words to "bell":
     "party" -- sim: 0.33197125792503357
     "dawned" -- sim: 0.32194608449935913
     "f" -- sim: 0.31368133425712585
     "curdken" -- sim: 0.30459749698638916
     "cry" -- sim: 0.30387431383132935
     "drop" -- sim: 0.2950055003166199
     "women" -- sim: 0.29164060950279236
     "stable" -- sim: 0.2907787561416626
     "talers" -- sim: 0.2869738042354584
     "planted" -- sim: 0.2802666425704956
Most similar words to "silver":
     "sense" -- sim: 0.4013127088546753
     "torn" -- sim: 0.3047332167625427
     "jewels" -- sim: 0.3018363416194916
     "dancing" -- sim: 0.30131450295448303
     "imagination" -- sim: 0.29833486676216125
     "nice" -- sim: 0.29233577847480774
     "bulk" -- sim: 0.2922404408454895
     "lamp" -- sim: 0.28793588280677795
     "sun" -- sim: 0.28562644124031067
     "smart" -- sim: 0.2854771018028259
Most similar words to "window":
     "house" -- sim: 0.4288135766983032
     "room" -- sim: 0.38202357292175293
     "door" 

In [35]:
#Step 9(c (iii))

similarity(10,100,2000)

Most similar words to "bell":
     "hunting" -- sim: 0.3386894762516022
     "immediate" -- sim: 0.31322407722473145
     "dawned" -- sim: 0.2904212474822998
     "disclaimer" -- sim: 0.2821723222732544
     "happening" -- sim: 0.27011704444885254
     "f" -- sim: 0.26597440242767334
     "grow" -- sim: 0.264279305934906
     "party" -- sim: 0.2610413134098053
     "mr" -- sim: 0.2588898539543152
     "ringing" -- sim: 0.25521984696388245
Most similar words to "silver":
     "miserable" -- sim: 0.37720009684562683
     "ear" -- sim: 0.33412835001945496
     "treasure" -- sim: 0.3237326741218567
     "flock" -- sim: 0.3183053731918335
     "advance" -- sim: 0.3124491572380066
     "howling" -- sim: 0.3085130751132965
     "easy" -- sim: 0.3064780831336975
     "whatsoever" -- sim: 0.3033634424209595
     "train" -- sim: 0.2900930643081665
     "sense" -- sim: 0.28422147035598755
Most similar words to "window":
     "house" -- sim: 0.39663082361221313
     "evidently" -- sim: 0.328763961

***Βήμα 9***

(γ)
(i) Γνωρίζουμε ότι όταν το μοντέλο μας πραγματοποιεί το training με μέγεθος παραθύρου ίσο με 3 τότε βρίσκουμε λέξεις που έχουν παρόμοια σύνταξη ενώ με μεγάλο παράθυρο βρίσκουμε λέξεις με παρόμοια σημασιολογία.  
Κάνοντας training λοιπόν το μοντέλο μας με παράθυρο ίσο με 10 και 15 θα έπρεπε να παρατηρήσουμε βελτίωση των αποτελεσμάτων για κάθε λέξη. Κάτι τέτοιο όμως δεν συμβαίνει. Υποθέτουμε ότι αυτό οφείλεται στο μικρού μεγέθους dataset στο οποίο γίνεται train το μοντέλο μας. 

(ii)